In [ ]:
from maicos import DensityPlanar, Diporder
import numpy as np
import os
import MDAnalysis as mda
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

In [ ]:
fontsize = 25
fontlegende = 22
font = {'family': 'sans', 'color':  'black', 
        'weight': 'normal', 'size': fontsize}
my_color_1 = np.array([0.090, 0.247, 0.560])
my_color_2 = np.array([0.235, 0.682, 0.639])
my_color_3 = np.array([1.000, 0.509, 0.333])
my_color_4 = np.array([0.588, 0.588, 0.588])
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})

In [ ]:
u = mda.Universe('solvated.data', 'prod.xtc')
total_frame = u.trajectory.n_frames

group_H2O = u.select_atoms('type 1 2')
group_CO2 = u.select_atoms('type 3 4')
group_CAR = u.select_atoms('type 5')
group_FLU = u.select_atoms('type 1 2 3 4')

In [ ]:
cpt = 0
dens_H2O = []
dens_CO2 = []
dens_CAR = []
dipo_H2O = []
dipo_CO2 = []
dipo_CAR = []
for mystop in np.arange(1, total_frame, 1):

    #number = '{0:04d}'.format(cpt)
    #name = 'figure.'+number+'.png'
    #cpt += 1

    #if os.path.exists(name) is False:

    # import universe and create groups
    #u = mda.Universe('solvated.data', 'prod.xtc')
    #u.transfer_to_memory(stop = mystop)
    #u.add_TopologyAttr('name', ['C']*np.sum(u.atoms.types == '3') 
    #                   + ['O', 'H1', 'H2']*np.sum(u.atoms.types == '1'))
    # density profile
    dens = DensityPlanar(atomgroups=[group_H2O, group_CO2, group_CAR], 
                            bin_width = 0.25, 
                            dens = "mass",
                            refgroup = group_FLU).run(start = mystop-1, stop = mystop)
    zcoor_1 = dens.results['bin_pos']
    dens_H2O.append(dens.results['profile_mean'].T[0])
    dens_CO2.append(dens.results['profile_mean'].T[1])
    dens_CAR.append(dens.results['profile_mean'].T[2])

    # diporder profile
    dipo = Diporder(atomgroups=[group_H2O, group_CO2, group_CAR], 
                    bin_width = 0.25, 
                    refgroup = group_FLU).run(start = mystop-1, stop = mystop)
    zcoor_2 = dipo.results['bin_pos']
    dipo_H2O.append(dipo.results['profile_mean'].T[0])
    dipo_CO2.append(dipo.results['profile_mean'].T[1])
    dipo_CAR.append(dipo.results['profile_mean'].T[2])

In [ ]:
for i in range(len(dens_CO2)):

    number = '{0:04d}'.format(i)
    name = 'figure.'+number+'.png'
    cpt += 1

    if i > 0:
        my_dens_H2O = np.mean(dens_H2O[:i], axis = 0)
        my_dens_CO2 = np.mean(dens_CO2[:i], axis = 0)
        my_dens_CAR = np.mean(dens_CAR[:i], axis = 0)
        my_dipo_H2O = np.mean(dipo_H2O[:i], axis = 0)
        my_dipo_CO2 = np.mean(dipo_CO2[:i], axis = 0)
        my_dipo_CAR = np.mean(dipo_CAR[:i], axis = 0)
    else:
        my_dens_H2O = np.array(dens_H2O[0])
        my_dens_CO2 = np.array(dens_CO2[0])
        my_dens_CAR = np.array(dens_CAR[0])
        my_dipo_H2O = np.array(dipo_H2O[0])
        my_dipo_CO2 = np.array(dipo_CO2[0])
        my_dipo_CAR = np.array(dipo_CAR[0])
    assert len(my_dens_H2O) == 230

    mypad = 10
    fig = plt.figure(figsize=(13,13))

    ax3 = plt.subplot(1, 2, 1)
    plt.plot(my_dens_H2O, zcoor_1/10, color=my_color_1, linewidth=4)
    plt.plot(my_dens_CO2, zcoor_1/10, color=my_color_2, linewidth=4)
    plt.plot(my_dens_CAR/4, zcoor_1/10, color=my_color_3, linewidth=4)
    plt.ylabel(r"$z$ (nm)", fontdict=font)
    plt.xlabel(r"density (amu/$\mathrm{\AA}$)", fontdict=font)
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.xticks([0, 0.4, 0.8, 1.2])
    plt.xlim(-0.1, 1.4)
    ax3.minorticks_on()
    ax3.tick_params('both', length=10, width=2, which='major', direction='in')
    ax3.tick_params('both', length=6, width=1.4, which='minor', direction='in')
    ax3.xaxis.set_ticks_position('both')
    ax3.yaxis.set_ticks_position('both')
    ax3.spines["top"].set_linewidth(2)
    ax3.spines["bottom"].set_linewidth(2)
    ax3.spines["left"].set_linewidth(2)
    ax3.spines["right"].set_linewidth(2)
    minor_locator_y = AutoMinorLocator(2)
    ax3.yaxis.set_minor_locator(minor_locator_y)
    minor_locator_x = AutoMinorLocator(2)
    ax3.xaxis.set_minor_locator(minor_locator_x)
    ax3.legend(frameon=False, fontsize=fontlegende, 
            loc='upper left', handletextpad=0.3,
            handlelength = 0.5, borderpad = 0.6, 
            labelspacing=0.6) 
    ax3.tick_params(axis='x', pad=10)

    ax3 = plt.subplot(1, 2, 2)
    plt.plot(my_dipo_H2O, zcoor_1/10, color=my_color_1, linewidth=4)
    plt.plot(my_dipo_CO2, zcoor_1/10, color=my_color_2, linewidth=4)
    plt.plot(my_dipo_CAR, zcoor_1/10, color=my_color_3, linewidth=4)
    plt.ylabel(r"$z$ (nm)", fontdict=font)
    plt.xlabel(r"$P_0$", fontdict=font)
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.xticks([-0.0015, 0, 0.0015])
    plt.xlim(-0.0016, 0.0016)
    ax3.minorticks_on()
    ax3.tick_params('both', length=10, width=2, which='major', direction='in')
    ax3.tick_params('both', length=6, width=1.4, which='minor', direction='in')
    ax3.xaxis.set_ticks_position('both')
    ax3.yaxis.set_ticks_position('both')
    ax3.spines["top"].set_linewidth(2)
    ax3.spines["bottom"].set_linewidth(2)
    ax3.spines["left"].set_linewidth(2)
    ax3.spines["right"].set_linewidth(2)
    minor_locator_y = AutoMinorLocator(2)
    ax3.yaxis.set_minor_locator(minor_locator_y)
    minor_locator_x = AutoMinorLocator(2)
    ax3.xaxis.set_minor_locator(minor_locator_x)
    ax3.legend(frameon=False, fontsize=fontlegende, 
            loc='upper left', handletextpad=0.3,
            handlelength = 0.5, borderpad = 0.6, 
            labelspacing=0.6) 
    ax3.tick_params(axis='x', pad=10)

    plt.savefig(name, bbox_inches = 'tight', pad_inches = 0.062)  
    #plt.show()